In [42]:
import pandas as pd
import re
import string
import numpy as np
from bs4 import BeautifulSoup
import nltk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [84]:
dataframes = {"tweet": pd.read_csv("datafix_promo.csv")}

In [85]:
dataframes["tweet"].drop_duplicates(inplace = True)

In [86]:
dataframes["tweet"].reset_index()

,index,Data,Promo,tweet
0,0,dah lama2 ni baru aku tahu OYO hotel ni hotel ...,0,dah lama2 ni baru aku tahu OYO hotel ni hotel ...
1,1,RT @nbalajiv: ‘Value’ migration series:\r\n\r\...,0,RT @nbalajiv: ‘Value’ migration series:\r\n\r\...
2,2,"RT @AB_BJP: Muslim Man Nekibur Zaman, son of f...",1,"RT @AB_BJP: Muslim Man Nekibur Zaman, son of f..."
3,3,RT @SpotTheDealUSA: $18 Nights #Groupon Discou...,1,RT @SpotTheDealUSA: $18 Nights #Groupon Discou...
4,5,【定期】格安のホテルや旅館を案内してたらOYOってところが多くなりました・・・泊まった製作者...,0,【定期】格安のホテルや旅館を案内してたらOYOってところが多くなりました・・・泊まった製作者...
5,9,$18 Nights #Groupon Discounted Rates OYO #Hote...,0,$18 Nights #Groupon Discounted Rates OYO #Hote...
6,11,"@OYO4U It's been more then 3 days, I'm expecti...",0,"@OYO4U It's been more then 3 days, I'm expecti..."
7,13,@OYO4U In late night when we check in hotel t...,0,@OYO4U In late night when we check in hotel t...
8,14,"Karena pengaruh iklan, akhirnya unduh shopback...",0,"Karena pengaruh iklan, akhirnya unduh shopback..."
9,22,"aku baru tau, oyo hotel founder dia orang indi...",0,"aku baru tau, oyo hotel founder dia orang indi..."


In [87]:
uri_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def stripTagsAndUris(x):
    if x:
        # BeautifulSoup on content
        soup = BeautifulSoup(x, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(uri_re, "", text)
    else:
        return ""

In [88]:
def removePunctuation(x):
    
    x = re.sub('RT','',x)
    # Lowercasing all words
    x = x.lower()
    # remove username
    x = re.sub('@[^\s]+','',x)
    # remove hastag
    x = re.sub('#[^\s]+','',x)
    # Removing non ASCII chars
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    # Removing (replacing with empty spaces actually) all the punctuations
    return re.sub("["+string.punctuation+"]", " ", x)

In [89]:
stops = set(stopwords.words("english"))
def removeStopwords(x):
    # Removing all the stopwords
    filtered_words = [word for word in x.split() if word not in stops]
    return " ".join(filtered_words)

In [90]:
from spellchecker import SpellChecker
 

def spell(words):
    checker = [SpellChecker().correction(words)]
    return checker

In [91]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]


def detoken(words):
    detok = [TreebankWordDetokenizer().detokenize(words)]
    return detok


In [94]:
# for df in dataframes.values():
#     df['tweet'] = df['tweet'].map(stripTagsAndUris)
#     df['tweet'] = df['tweet'].map(removePunctuation)
#     df['tweet'] = df['tweet'].map(removeStopwords)
#     df['tweet'] = df['tweet'].apply(spell)
    
    
for df in dataframes.values():
    df['Data'] = df['Data'].map(stripTagsAndUris)
    df['Data'] = df['Data'].map(removePunctuation)
    df['Data'] = df['Data'].map(removeStopwords)
#     df['Data'] = df['Data'].apply(spell)
    df['Data'] = df['Data'].apply(lemmatize_text)
    df['Data'] = df['Data'].apply(detoken)

C:\Users\Mon\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/Yh7Yf718Wa" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Mon\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/3tP0Ys3q4e" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Mon\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/9QhyMRTwze" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beaut

In [95]:
df['Data']

0       [dah lama2 ni baru aku tahu oyo hotel ni hotel...
1       [value migration series kid market cap oyo gre...
2       [muslim man nekibur zaman son former minister ...
3          [18 night discounted rate oyo w date starting]
5                                                   [oyo]
9          [18 night discounted rate oyo w date starting]
11      [3 day expecting call resolution query seems h...
13      [late night check hotel hotel management say h...
14      [karena pengaruh iklan akhirnya unduh shopback...
22      [aku baru tau oyo hotel founder dia orang indi...
23      [melanjutkan permintaan netizen present hotel ...
27      [hope still remember statement made stil hotel...
28                  [couple enters oyo hotel hotel staff]
29      [value migration series indian stock kid marke...
32      [check dekat hotel oyo memang nasib la kena raid]
34      [ritesh agarwal founder ceo world third larges...
35      [ritesh agarwal founder ceo world third larges...
36      [value

In [97]:
dataframes["tweet"] = df[df.Promo != 1]

for name, df in dataframes.items():
    # Saving to file
    df.to_csv(name + "_datafix.csv", index=False)